In [ ]:
from keras.backend import image_data_format, clear_session
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from keras.layers import Activation, BatchNormalization, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Reshape
from keras.models import Sequential
from keras.optimizers import SGD
from keras.utils import to_categorical

from sklearn import datasets
from sklearn.cross_validation import train_test_split

from utils.ferloader import load_fer

from random import uniform

# Constants
IMAGE_WIDTH = 48
IMAGE_HEIGHT = 48
EMOTIONS = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
DATA_FORMAT = image_data_format()

# Prepare data
data, target = load_fer("data/fer2013.npz")
data_train, data_test, target_train, target_test = \
  train_test_split(data, target, test_size=0.2, random_state=7, stratify=target)
_, n_features = data.shape

# Prepare targets
onehot_train = to_categorical(target_train)
onehot_test = to_categorical(target_test)

while True:
    # Build model
    model = Sequential()
    if DATA_FORMAT == 'channels_first':
        model.add(Reshape((1, IMAGE_HEIGHT, IMAGE_WIDTH), input_shape=(n_features,)))
    else:
        model.add(Reshape((IMAGE_HEIGHT, IMAGE_WIDTH, 1), input_shape=(n_features,)))
    model.add(Conv2D(64, kernel_size=(5, 5), padding='same'))
    if DATA_FORMAT == 'channels_first':
        model.add(BatchNormalization(axis=1))
    else:
        model.add(BatchNormalization(axis=3))
    model.add(Activation(activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(5, 5), padding='same'))
    if DATA_FORMAT == 'channels_first':
        model.add(BatchNormalization(axis=1))
    else:
        model.add(BatchNormalization(axis=3))
    model.add(Activation(activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(len(EMOTIONS), activation='softmax'))

    LR = 0.1 * pow(0.1, uniform(0, 3))
    MOMENTUM = uniform(0.1, 0.5)
    DECAY = pow(0.1, uniform(2, 6))

    print("Learning rate = {:.4f}, Momentum = {:.3f}, Decay = {:.6f}".format(LR, MOMENTUM, DECAY))
    
    # Compile model
    momentum_opt = SGD(lr=LR, momentum=MOMENTUM, decay=DECAY, nesterov=True)
    model.compile(
        loss='categorical_crossentropy',
        optimizer=momentum_opt,
        metrics=['accuracy']
    )

    # Fit model
    callbacks_list = [
        ModelCheckpoint('./checkpoints/best_lr={:.4f}_m={:.3f}_decay={:.6f}.hdf5'.format(LR, MOMENTUM, DECAY), monitor='val_loss', verbose=1, save_best_only=True, mode='min'),
        TensorBoard(log_dir='./logs/lr={:.4f}_m={:.3f}_decay={:.6f}'.format(LR, MOMENTUM, DECAY)),
        EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3, verbose=1, mode='auto')
    ]
    model.fit(data_train, onehot_train, epochs=50, batch_size=32, validation_split=0.1, callbacks=callbacks_list)

    # Clean up after training
    del model
    clear_session()

Learning rate = 0.0032, Momentum = 0.186, Decay = 0.004689
Train on 25838 samples, validate on 2871 samples
Epoch 1/50
25838/25838 [==============================] - 47s 2ms/step - loss: 1.7494 - acc: 0.3198 - val_loss: 1.5935 - val_acc: 0.3817

Epoch 00001: val_loss improved from inf to 1.59350, saving model to ./checkpoints/best_lr=0.0032_m=0.186_decay=0.004689.hdf5
Epoch 2/50
25838/25838 [==============================] - 54s 2ms/step - loss: 1.6037 - acc: 0.3804 - val_loss: 1.5551 - val_acc: 0.3950

Epoch 00002: val_loss improved from 1.59350 to 1.55505, saving model to ./checkpoints/best_lr=0.0032_m=0.186_decay=0.004689.hdf5
Epoch 3/50
25838/25838 [==============================] - 55s 2ms/step - loss: 1.5631 - acc: 0.4024 - val_loss: 1.5318 - val_acc: 0.4058

Epoch 00003: val_loss improved from 1.55505 to 1.53182, saving model to ./checkpoints/best_lr=0.0032_m=0.186_decay=0.004689.hdf5
Epoch 4/50
25838/25838 [==============================] - 54s 2ms/step - loss: 1.5399 - acc: 0.

In [ ]:
# Evaluate best model
model.load_weights("./checkpoints/best_weights.hdf5")
results = model.evaluate(data_test, onehot_test, batch_size=len(data_test))
print("\n[!] Evaluation results:")
print("{0}: {2:.3f}, {1}: {3:.3f}".format(*model.metrics_names, *results))